In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
sns.set()
import requests
import json
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.dates import MO
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

In [13]:
# sometimes this API is down 
URL = 'https://covidtracking.com/api/states/daily'
r = requests.get(URL)
sd = pd.read_json(json.dumps(r.json()))

# convert floats to dates
# sd['date'] = pd.to_datetime(sd['date'], format='%Y%m%d')

def get_fit(state, data):
    fit_data = data
    # deal with dates and drop nulls
    fit_data.index = pd.to_datetime(fit_data['date'], format='%Y%m%d')
    fit_data = data['positive']
    data_threshold_cases = 1
    fit_data[fit_data < data_threshold_cases] = np.nan
    fit_data = fit_data.dropna()
    fit_data = fit_data[-5:]
    # return nulls if less than two days data
    if len(fit_data) < 2:
        return {'state': state, 'beta': None}
    # prepare for log fit
    days = (fit_data.index - fit_data.index[0]) / np.timedelta64(1, 'D')
    log_fd = np.log(fit_data.values)
    # perform fit
    beta, log_initial = np.polyfit(days, log_fd, 1)
    # report time constant
    return {'state': state, 'beta': beta, 'time_constant': 1/beta, 'num_points': len(fit_data)}
    
fit_data = []
for state, data in sd.groupby('state'):
    fit_data.append(get_fit(state, data))
    
time_constants = pd.DataFrame(fit_data)
time_constants['doubling_time'] = time_constants['time_constant'] * np.log(2)
time_constants = time_constants.sort_values('time_constant')
time_constants

<ipython-input-13-cb1d2ca9395b>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fit_data[fit_data < data_threshold_cases] = np.nan
<ipython-input-13-cb1d2ca9395b>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fit_data.append(get_fit(state, data))
<ipython-input-13-cb1d2ca9395b>:27: RuntimeWarning: divide by zero encountered in double_scalars
  return {'state': state, 'beta': beta, 'time_constant': 1/beta, 'num_points': len(fit_data)}


,state,beta,time_constant,num_points,doubling_time
20,LA,0.980610,1.019774,5.0,0.706853
1,AL,0.870748,1.148438,5.0,0.796036
24,MI,0.813245,1.229642,5.0,0.852323
37,NY,0.696409,1.435937,5.0,0.995316
54,WV,0.667456,1.498226,5.0,1.038491
34,NJ,0.658755,1.518015,5.0,1.052208
47,TX,0.623832,1.602994,5.0,1.111111
8,DC,0.599146,1.669041,5.0,1.156891
19,KY,0.595064,1.680491,5.0,1.164827
28,MS,0.588610,1.698917,5.0,1.177599
